### Azure AI Search Multi-Modal RAG - Simple Image Verbalisation Lab

![simple_image_verbalisation.png](./Assets/simple_image_verbalisation.png)

### Installing Required Libraries and Utilities

In [ ]:
%pip install openai requests python-dotenv

### Setting Up the Environment

In [ ]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_API_KEY")

### Creating the Azure OpenAI Client

In [ ]:
from openai import AzureOpenAI

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")

azure_openai_client = AzureOpenAI(
    api_key=azure_openai_key,
    api_version="2024-02-15-preview",
    azure_endpoint=azure_openai_endpoint
)

### Creating the Embedding Generator Function

In [ ]:
def generate_embeddings(client, text):
    embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
    
    response = client.embeddings.create(
        input=text,
        model = embedding_model
    )
    
    embeddings=response.model_dump()
    return embeddings['data'][0]['embedding']

In [ ]:
user_query = "can you tell me something about the sustainability initiatives at BMW?"
vectorised_user_query = generate_embeddings(azure_openai_client, user_query)
print(vectorised_user_query)

In [ ]:
context=[]

### Sending API Call to the Search Index

In [ ]:
import requests
import json


url = f"{service_endpoint}/indexes/{index_name}/docs/search?api-version=2023-11-01"
    
headers = {
        "Content-Type": "application/json",
        "api-key": key
    }
    
body =   {
        "count": True,
        "select": "document_title, content_text, locationMetadata, image_document_id",
        "vectorQueries": [
            {
                "vector": vectorised_user_query,
                "k": 10,
                "fields": "content_embedding",
                "kind": "vector"
            }
        ]
    }
    
response = requests.post(url, headers=headers, data=json.dumps(body))
documents = response.json()['value']

for doc in documents:
    context.append(dict(
        {
            "document_title": doc['document_title'],
            "chunk": doc['content_text'],
            "score": doc['@search.score'],
            "locationMetadata": doc['locationMetadata'] if 'locationMetadata' in doc else None,
            "image_document_id": doc['image_document_id'] if 'image_document_id' in doc else None
        }
    ))
    
for doc in context:
    print(doc)

### Calling GPT Engine for Summarisation

In [ ]:
system_prompt = f""""You are meant to behave as a RAG chatbot that derives its context from a database stored in Azure AI Search Solution.
please answer strictly from the context from the database provided and if you dont have an answer please politely say so. dont include any extra 
information that is not in the context and dont include links as well.
the context passed to you will be in the form of a pythonic list with each object in the list having the following structure:

{{
    "document_title": "the title of the document",
    "chunk": "the chunk of text from the document",
    "score": "the score of the match based on cosine similarity",
    "locationMetadata": "the location metadata if available, else None",
    "image_document_id": "the image document id if available, else None",
}}

the pythonic list contains best 10 matches to the user query based on cosine similarity of the embeddings of the user query and the review descriptions.
please structure your answers in a very professional manner and in such a way that the user does not get to know that its RAG working under the hood
and its as if they are talking to a human. """

user_prompt = f""" the user query is: {user_query}
the context is : {context}"""

chat_completions_response = azure_openai_client.chat.completions.create(
    model = os.getenv("AZURE_OPENAI_CHAT_COMPLETIONS_MODEL"),
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.7
)

print(chat_completions_response.choices[0].message.content)